# Microbial Genomics: Lab 4
## Topic: Bash, Genome Assembly & Comparison
#### Tools used: Fastp, FastQC, Samtools, SPAdes, Bandage

In [ ]:
# Compatibility code for running in Google Colab - please run but free to ignore
import os

if "colab" in str(get_ipython()):
    from google.colab import drive

    drive.mount("/content/drive")
    os.chdir("/content/drive/My Drive/microbial_genomics_labs/labs")

In [ ]:
!pip install biopython
!apt install samtools spades fastp fastqc minimap2 -y

## Part A: Lab Exercises (10 pts)
### Exercise 1: FastQ Basics (2 pts)
The FastQ format is used to contain raw DNA reads that come directly from a sequencer; these reads need to be processed during DNA assembly in order to be compiled into a usable format, such as the `fasta` files we have dealt with so far. Unlike our `fasta` files, `fastq` files have no context on where they came from or what organism they belong to- it's up to us to figure this out via assembly. Fortunately, there are many excellent tools to help us!

The anatomy of a `fastq` file differs from a `fasta` file in several key ways:
* Each DNA strand (i.e., forward and reverse) will often have a separate set of reads; these are usually contained in two separate files, but can sometimes be "interleaved" in a single file (in the case of paired-end sequencing)
* FastQ files are typically much larger than their FastA counterparts; this is because it takes many reads to create a single high-quality sequence
* Each read in a FastQ typically contains four lines; the first begins with an @ symbol and a sequence ID, followed by the sequence itself, a line with a + symbol, and finally, a line describing the quality of the read. For example:

```
@SEQ_ID
GATTTGGGGTTCAAAGCAGTATCGATCAAATAGTAAATCCATTTGTTCAACTCACAGTTT
+
!''*((((***+))%%%++)(%%%%).1***-+*''))**55CCF>>>>>>CCCCCCC65
```

**Open the file `lab4/R64drd.fastq` in a text editor and answer the following questions in comments below:**
1. What can you tell about the sequences in this file?
2. Look at the first sequence and the 5th sequence and look at the quality lines. Use [this page](https://en.wikipedia.org/wiki/FASTQ_format) to assess the quality of the read. Are they generally high-quality? How does the quality change over the course of the read? Are the 1st and 5th sequence of similar quality?
3. Is this file interleaved or single-stranded? Why do you think so?

In [ ]:
# Exercise 1

### Exercise 2: FastQ Coverage and QC (2 pts)
One of the reasons FastQ files are much larger than the genomes they represent is because high-quality reads typically have very high "coverage" of the genome, meaning that for each base in the full genome, a FastQ file should have upwards of 300 reads (i.e., 300x coverage); that is, for E. Coli, which has a genome size of 4.6Mb, there can be more than 1.3 _billion_ bases.

Because even modestly sized genomes require FastQ files that are several GB, we'll be working with plasmid sequences, which are smaller (~2000 bp). In this exercise, we'll be performing QC (quality control) for these sequences.

`fastp` is a general-purpose FastQ processing toolkit that includes QC, trimming, filtering, deduplication, and other useful functionality. For the most part, we will be concerned with qc (removing low-quality reads) and trimming (removing adapter sequences) for our pipelines. `fastqc` is another tool used to generate nice QC reports.

In [ ]:
# run FastP on our fastq files; discard reads of quality <25, trim the adapters, and generate a report
! fastp -n 25 -h lab4/fastp_report.html -i lab4/raw_fastq/r1.fastq.gz -I lab4/raw_fastq/r2.fastq.gz -o lab4/trimmed_fastq/r1.trimmed.fastq.gz -O lab4/trimmed_fastq/r2.trimmed.fastq.gz

# Create fastqc_out directory
! mkdir lab4/fastqc_out

# run fastQC on our reads and save in the fastqc_out directory
! fastqc lab4/trimmed_fastq/*.fastq.gz -o lab4/fastqc_out

**Using the analysis performed above, answer the following questions:**
1. If we know that the plasmid that was sequenced has a length of 120826 bp, and we used a sequencer with paired read length of 150, what is the estimated coverage for this fastq file?
2. Open one of the fastQC HTML files. Based on this [QC tutorial](https://rtsf.natsci.msu.edu/genomics/technical-documents/fastqc-tutorial-and-faq.aspx), are our reads high quality or not? Are there any graphs that are particularly interesting?
3. Open up `fastp_report.html`. How many reads did we lose by performing QC and trimming? Does this seem like a large number?
4. Are both reads approximately the same in terms of quality and resulting size? If not, what is different?

In [ ]:
# Exercise 2

### Exercise 3: Using Command Line Tools and Reference Genomes (2 pts)
You may have noticed that we used a `!` above to run the `fastp` command. We're actually running commands on the command line when we do this; as we use more tools throughout this course, you'll notice that although many of them are written in Python, they exist as standalone executables that need to be called via Bash. This is because most Bioinformatics pipelines are built using pipelines of many different programs linked together, and creating such pipelines is a relatively natrual process within a shell environment.

_If you haven't used Bash before, now might be a good time to review tutorials such as [this one](https://www.freecodecamp.org/news/bash-scripting-tutorial-linux-shell-script-and-command-line-for-beginners/)._

From Jupyter/Colab, you can run any command that you'd have access to from your compute environment by using `!`. `samtools` is a common suite of tools to work with SAM (non-binary) and BAM (binary) files, which are raw versions of aligned reads. Below, we'll use `samtools` to start exploring our `fastq` files and calculate coverage programmatically. The steps we'll take are:
1. Align our FastQ reads to a known reference genome
2. Correct any mate-pair issues that happened during alignment
3. Sort and index the resulting SAM file for fast downstream operations
5. Check the coverage of the resulting SAM file

*Note: this is a subset of a very common samtools workflow, which is covered in more depth [here](https://www.htslib.org/workflow/fastq.html)*.

In [ ]:
# run minimap to align our fastq reads to the reference genome
! minimap2 -a -x sr lab4/reference/R64_ref.fasta lab4/trimmed_fastq/*1.trimmed.fastq.gz lab4/trimmed_fastq/*2.trimmed.fastq.gz -o lab4/aligned/R64.sam

# fix the mate-pairs that may have been effected by alignment
! samtools fixmate -O sam lab4/aligned/R64.sam lab4/aligned/R64_fixmate.sam

# Sort the SAM file- this is required for coverage calculation
! samtools sort -l 9 -o lab4/aligned/R64_sort.bam -T lab4/tmp/R64 lab4/aligned/R64_fixmate.sam

# calculate coverage and depth of the SAM file
! samtools depth lab4/aligned/R64_sort.bam > lab4/aligned/coverage.txt


**Use the files generated in the above cells to answer the following questions:**
1. Open one of the generated SAM files. How is it different from the raw fastq files?
2. What's different between the SAM and BAM files?
3. Take a look at the `coverage.txt` file. What is it telling us?

*Note: you may want to delete some of the files generated above after you're done with these questions if you're running low on disk space!*

In [ ]:
# Exercise 3

### Exercise 4: Piping & I/O (2 pts)
Above, we ran several lines, one at a time, in order to produce an output. This worked, but we ended up with several (large) intermediate files. Often, we don't need to use these files, so to get around having to write them out and then read them back in, we can use bash _piping_. Essentially, this just means we take the output from one command and feed it into the next command without ever writing it into a file. This will be useful when we begin building larger pipelines below.

Let's take the commands from Exercise 3 and put them into a single command. We'll use `%%bash` at the top of the cell to turn the entire cell into a bash interpreter, instead of using `!`. We'll also use the `\` character to continue lines, for readability.

_Note: One important operator we're not using explicitly here is `>`, which is used to redirect output. Using `>` at any stage in this pipeline would direct the output of the preceding command to a file, i.e., writing `cat lab4/reference/R64_ref.fasta > lab4/reference/R64_ref2.fasta` would take the contents of `R64_ref.fasta` and write them into `R64_ref2.fasta`. You may need to use `>` in your homework assignment, so read up on it in the bash tutorial above if you feel the need._

In [ ]:
%%bash
minimap2 -a -x sr lab4/reference/R64_ref.fasta lab4/trimmed_fastq/*1.trimmed.fastq.gz lab4/trimmed_fastq/*2.trimmed.fastq.gz  | \
samtools fixmate -m - - | \
samtools sort -T lab4/tmp/R64 - | \
samtools depth - > lab4/aligned/coverage_piped.txt

**Based on the code and results above, answer the following questions:**
1. What does the `-` character do in the code above?
2. What files were created in this workflow, compared to the one in Exercise 3?
3. Are there any potential issues with this type of piped workflow?

In [ ]:
# Exercise 4

### Exercise 5: SPAdes (2 pts)
Now that we have some familiarity with Bash, lets assemble our FastQ files! De novo assembly is used if a reference is unknown or non-existant; it attempts to build scaffolds from the reads, and assembles these scaffolds into a full genome. When the reference genome is known, we perform a different type of assembly, known as mapping assembly or alignment. `samtools` is actually a low-level aligner, and can perform this operation, but many other more sophisticated programs exist- we'll cover those in next week's lab.

For this laboratory, we'll use `spades` for de novo assembly. Others exist, but SPAdes is well-known and used often for short-read bacterial genome assembly. Note that **although we know the reference genome for our reads, we will pretend we do not for this exercise**; this is the basis of de novo assemblers. Although some include options for "reference-guided" assembly, they operate largely on the basis that we know very little about our input reads.

_Note: on Colab, this cell takes ~2 minutes to run_

In [ ]:
! spades.py -1 lab4/trimmed_fastq/r1.trimmed.fastq.gz -2 lab4/trimmed_fastq/r2.trimmed.fastq.gz -o lab4/spades_output

That's all it takes! As you can see, SPAdes has a fairly simple command line interface.

#### Use the output from the above command to answer the following questions:
1. Take a look at the [SPAdes documentation](https://github.com/ablab/spades); do you see any program options that would let us include a reference genome, if we suspected it might be related to our reads?
2. Open the log file found at `lab4/spades_output/spades.log`. This log file contains all the program output from our assembly. What is the kmer size that was used by SPAdes to create the final assembly?
3. Our final assembly is contained in `scaffolds.fasta`. What size is this file, compared to `R64_ref.fasta`, the actual reference? Based on the coverage we calculated earlier, is this about right?
4. Note that your assembly has multiple contigs. Why do you think that is? Blast the first contig against the reference - how good do you think your assembly is?

In [ ]:
# Exercise 5

## Part B: Homework

### Question 1: Assembly pipeline (10 pts)
The main objective of this homework will be to build your own basic assembly pipeline using the tools we've discussed in this lab. You'll be assembling some reads from an unknown organism using de novo assembly, and then answering some questions about the organism based on the result. Your program can be written in an external shell script, or in Jupyter/Colab, but must complete the following steps:
1. Perform QC and trimming on the raw FastQ reads `HW4q1_r1.fastq` and `HW4q1_r2.fastq`, discarding any reads that fail to meet a quality threadhold of N=20
2. Calculate the *average* coverage of your resulting trimmed reads. This should be done by averaging all individual positions returned from the coverage file.
3. Assemble the trimmed reads using SPAdes
4. Remove up any temporary files, but make sure to keep the SPAdes scaffold (`scaffolds.fasta`), the fastp/fastqc quality report, and the assembly graph (`assembly_graph.fastg`)
5. Locally BLAST your assembly against the `silva_species_ID.fasta` database (included as a fasta file) and return the presumed taxonomy of your genome. Open up the silva .fasta file to investigate what information would be useful to return.

If you choose to write your script externally, you should still run it in the cell below. Make sure all results are placed into the `lab4/results/` folder at the end of your code.

In [ ]:
# Question 1

### Question 2: Read refinement (6 pts)

Pipelines are often iterative, meaning that we may need to perform multiple runs after changing some parameters. Open the FastP or FastQC report that you generated above. Are the reads that you used for steps 2-4 high-quality? Comment on any steps you would consider taking to refine the reads further in the cell below.

In [ ]:
# Question 2

### Question 3: SILVA commentary (6 pts)

Using the results from your SILVA BLAST search, answer the following:
* What types of hits showed up in your BLAST search? Were all of the organisms from the same species?
* How good was the top hit that you downloaded? If you think the match was good, describe why. If not, describe what you think went wrong- does a better hit exist in a different database? Was the assembly bad?

In [ ]:
# Question 3

### Question 4: Bandage (8 pts)

As we discussed earlier, de novo assembly is generally difficult to evaluate. One way of telling how "good" our assembly is, is checking the De Bruijn graph. If our assembly is good, it will have a single contig, and the graph will be a fully connected "circle". If not, there will be (potentially many) disconnected or discontiguous loops. (*Note: a bad assembly can also have a "good"  graph- but a good assembly will never have a bad graph!*)

We'll use the `bandage` tool to view our graph. Go to the [Bandage GitHub page](https://rrwick.github.io/Bandage/) and download/install the tool. Once it is installed, run it and open up the `assembly_graph.fastg` file. Describe the graph you see below; is it contiguous? Are there extra "pieces" of the genome outside the main graph? Based on your knowledge, is our assembly of good quality?

In [ ]:
# Question 4